In [ ]:
# saving only segmented images
import cv2
import numpy as np
import glob, os
from ultralytics import YOLO

lines_model = 'Models/Lanes_seg_v2.pt'
model = YOLO(lines_model)

counter = 0
images_src_dir = 'Data/Raw_images'
images_list = [i for i in glob.glob(images_src_dir + os.sep + "*.jpg")]
output_dir = 'Data/Detected/images'

os.makedirs(output_dir, exist_ok=True)

colors = [
    (255, 0, 0),  # Red
    (255, 0, 0),  # Green
    (0, 0, 255),  # Blue
    (255, 255, 0),  # Cyan
    (255, 0, 255),  # Magenta
    (0, 255, 255),  # Yellow
]

for imagefile in images_list:
    image_name = os.path.basename(imagefile)
    image_path = imagefile

    img = cv2.imread(image_path)
    img_height, img_width = img.shape[:2]  # Get image dimensions

    results = model(img, conf=0.3)

    blended_image = img.copy()  # Create a copy of the original image for blending
    masks_detected = False  # Flag to check if any masks were detected

    combined_mask = np.zeros((img_height, img_width, 3), dtype=np.uint8)  # Initialize combined mask with 3 channels

    for idx, result in enumerate(results):
        if result.masks is not None:
            masks = result.masks.data  # Segmentation masks
            classes = result.boxes.cls.cpu().numpy().astype(int)  # Get class IDs
            class_names = result.names  # Get class names

            for i, mask in enumerate(masks):
                masks_detected = True  # Set flag to True if any mask is detected
                mask = (mask.cpu().numpy() * 255).astype("uint8")  # Move tensor to CPU and convert to binary
                mask_resized = cv2.resize(mask, (img_width, img_height), interpolation=cv2.INTER_NEAREST)
                
                # Create a light bright green mask
                light_bright_green = np.zeros((img_height, img_width, 3), dtype=np.uint8)
                light_bright_green[:, :] = (0, 255, 0)  # Light bright green color
                mask_colored = cv2.bitwise_and(light_bright_green, light_bright_green, mask=mask_resized)
                
                combined_mask = cv2.add(combined_mask, mask_colored)  # Combine masks

                class_name = class_names[classes[i]]
                color = colors[classes[i] % len(colors)]  # Get color for the class

                x1, y1, x2, y2 = result.boxes.xyxy[i].cpu().numpy().astype(int)
                center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

                cv2.putText(
                    blended_image,
                    class_name,
                    (center_x, center_y),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    color,
                    2,
                    cv2.LINE_AA,
                )

    if masks_detected:
        blended_image = cv2.addWeighted(blended_image, 0.8, combined_mask, 0.2, 0)  # Lower opacity of the mask
        output_path = os.path.join(output_dir, f"{os.path.splitext(image_name)[0]}_result.jpg")
        cv2.imwrite(output_path, blended_image)
        
        counter += 1
    else:
        print(f"No masks detected for image: {image_name}")

In [ ]:
# saving segmented images along with bbox
import cv2
import numpy as np
import glob, os
from ultralytics import YOLO

lines_model = 'Models/Lanes_seg_v2.pt'
model = YOLO(lines_model)

counter = 0
images_src_dir = 'Data/Raw_images'
images_list = [i for i in glob.glob(images_src_dir + os.sep + "*.jpg")]
output_dir = 'Data/Detected/images'

os.makedirs(output_dir, exist_ok=True)

colors = [
    (255, 0, 0),  # Red
    (255, 0, 0),  # Green
    (0, 0, 255),  # Blue
    (255, 255, 0),  # Cyan
    (255, 0, 255),  # Magenta
    (0, 255, 255),  # Yellow
]

for imagefile in images_list:
    image_name = os.path.basename(imagefile)
    image_path = imagefile

    img = cv2.imread(image_path)
    img_height, img_width = img.shape[:2]  # Get image dimensions

    results = model(img, conf=0.3)

    blended_image = img.copy()  # Create a copy of the original image for blending
    masks_detected = False  # Flag to check if any masks were detected

    combined_mask = np.zeros((img_height, img_width, 3), dtype=np.uint8)  # Initialize combined mask with 3 channels

    yolo_format_lines = []  # List to store YOLOv8 format lines

    for idx, result in enumerate(results):
        if result.masks is not None:
            masks = result.masks.data  # Segmentation masks
            classes = result.boxes.cls.cpu().numpy().astype(int)  # Get class IDs
            class_names = result.names  # Get class names

            for i, mask in enumerate(masks):
                masks_detected = True  # Set flag to True if any mask is detected
                mask = (mask.cpu().numpy() * 255).astype("uint8")  # Move tensor to CPU and convert to binary
                mask_resized = cv2.resize(mask, (img_width, img_height), interpolation=cv2.INTER_NEAREST)
                
                # Create a light bright green mask
                light_bright_green = np.zeros((img_height, img_width, 3), dtype=np.uint8)
                light_bright_green[:, :] = (0, 255, 0)  # Light bright green color
                mask_colored = cv2.bitwise_and(light_bright_green, light_bright_green, mask=mask_resized)
                
                combined_mask = cv2.add(combined_mask, mask_colored)  # Combine masks

                class_name = class_names[classes[i]]
                color = colors[classes[i] % len(colors)]  # Get color for the class

                x1, y1, x2, y2 = result.boxes.xyxy[i].cpu().numpy().astype(int)
                center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

                cv2.putText(
                    blended_image,
                    class_name,
                    (center_x, center_y),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    color,
                    2,
                    cv2.LINE_AA,
                )

                # Convert to YOLOv8 format
                bbox_width = x2 - x1
                bbox_height = y2 - y1
                yolo_center_x = (x1 + x2) / 2 / img_width
                yolo_center_y = (y1 + y2) / 2 / img_height
                yolo_width = bbox_width / img_width
                yolo_height = bbox_height / img_height

                # Get the segmentation points
                contours, _ = cv2.findContours(mask_resized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                for contour in contours:
                    points = contour.reshape(-1, 2)
                    normalized_points = [(x / img_width, y / img_height) for x, y in points]
                    formatted_points = " ".join([f"{x} {y}" for x, y in normalized_points])
                    yolo_format_lines.append(f"{classes[i]} {formatted_points}")

    if masks_detected:
        blended_image = cv2.addWeighted(blended_image, 0.8, combined_mask, 0.2, 0)  # Lower opacity of the mask
        output_path = os.path.join(output_dir, f"{os.path.splitext(image_name)[0]}.jpg")
        # output_path = os.path.join(output_dir, f"{os.path.splitext(image_name)[0]}_result.jpg")
        cv2.imwrite(output_path, blended_image)

        # Save YOLOv8 format annotations
        txt_output_path = os.path.join(output_dir, f"{os.path.splitext(image_name)[0]}.txt")
        with open(txt_output_path, 'w') as f:
            f.write("\n".join(yolo_format_lines))
        
        counter += 1
    else:
        print(f"No masks detected for image: {image_name}")

In [ ]:
# shifted segmented Contours
import cv2
import numpy as np
import glob, os
from ultralytics import YOLO

lines_model = 'Models/Lanes_seg_v2.pt'
model = YOLO(lines_model)

counter = 0
images_src_dir = 'Data/Raw_images'
images_list = [i for i in glob.glob(images_src_dir + os.sep + "*.jpg")]
output_dir = 'Data/Detected/images'

os.makedirs(output_dir, exist_ok=True)

colors = [
    (255, 0, 0),  # Red
    (255, 0, 0),  # Green
    (0, 0, 255),  # Blue
    (255, 255, 0),  # Cyan
    (255, 0, 255),  # Magenta
    (0, 255, 255),  # Yellow
]

for imagefile in images_list:
    image_name = os.path.basename(imagefile)
    image_path = imagefile

    img = cv2.imread(image_path)
    img_height, img_width = img.shape[:2]  # Get image dimensions

    results = model(img, conf=0.3)

    blended_image = img.copy()  # Create a copy of the original image for blending
    masks_detected = False  # Flag to check if any masks were detected

    for idx, result in enumerate(results):
        if result.masks is not None:
            masks = result.masks.data  # Segmentation masks
            classes = result.boxes.cls.cpu().numpy().astype(int)  # Get class IDs
            class_names = result.names  # Get class names

            for i, mask in enumerate(masks):
                masks_detected = True  # Set flag to True if any mask is detected
                mask = (mask.cpu().numpy() * 255).astype("uint8")  # Move tensor to CPU and convert to binary
                mask_resized = cv2.resize(mask, (img_width, img_height), interpolation=cv2.INTER_NEAREST)
                
                # Find contours of the mask
                contours, _ = cv2.findContours(mask_resized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                if contours:
                    # Approximate contours to reduce the number of points
                    approx_contours = [cv2.approxPolyDP(cnt, epsilon=1.0, closed=True) for cnt in contours]
                    
                    for cnt in approx_contours:
                        for j in range(len(cnt)):
                            point = cnt[j][0]
                            distance_from_center = abs(point[0] - img_width / 2)
                            max_distance = img_width / 2
                            shift_factor = (distance_from_center / max_distance) * 38  # Gradient shift

                            if point[0] < img_width / 2:
                                # Point is on the left side of the image
                                shift_x = shift_factor
                            else:
                                # Point is on the right side of the image
                                shift_x = -shift_factor

                            shift_y = 0  # No vertical shift needed, adjust if necessary
                            M = np.float32([[1, 0, shift_x], [0, 1, shift_y]])
                            shifted_point = cv2.transform(np.array([[point]], dtype=np.float32), M)[0][0]
                            cnt[j][0] = shifted_point

                        # Connect endpoints of the contours to handle discontinuities
                        for j in range(len(cnt) - 1):
                            cv2.line(blended_image, tuple(cnt[j][0]), tuple(cnt[j + 1][0]), (0, 255, 0), 1)
                        cv2.line(blended_image, tuple(cnt[-1][0]), tuple(cnt[0][0]), (0, 255, 0), 1)
                
                class_name = class_names[classes[i]]
                color = colors[classes[i] % len(colors)]  # Get color for the class

                cv2.putText(
                    blended_image,
                    class_name,
                    (point[0], point[1]),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    color,
                    2,
                    cv2.LINE_AA,
                )

    if masks_detected:
        output_path = os.path.join(output_dir, f"{os.path.splitext(image_name)[0]}_result.jpg")
        cv2.imwrite(output_path, blended_image)
        
        counter += 1
    else:
        print(f"No masks detected for image: {image_name}")

In [ ]:
# filtering overlapping boxes
import cv2
import numpy as np
import glob, os
from ultralytics import YOLO

def calculate_iou(box1, box2):
    x1, y1, x2, y2 = box1
    x1_, y1_, x2_, y2_ = box2

    xi1 = max(x1, x1_)
    yi1 = max(y1, y1_)
    xi2 = min(x2, x2_)
    yi2 = min(y2, y2_)

    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2_ - x1_) * (y2_ - y1_)

    union_area = box1_area + box2_area - inter_area

    iou = inter_area / union_area
    return iou

lines_model = 'Models/Lanes_seg_v2.pt'
model = YOLO(lines_model)

counter = 0
images_src_dir = 'Data/Raw_images'
images_list = [i for i in glob.glob(images_src_dir + os.sep + "*.jpg")]
output_dir = 'Data/Detected/images'

os.makedirs(output_dir, exist_ok=True)

colors = [
    (255, 0, 0),  # Red
    (255, 0, 0),  # Green
    (0, 0, 255),  # Blue
    (255, 255, 0),  # Cyan
    (255, 0, 255),  # Magenta
    (0, 255, 255),  # Yellow
]

overlap_threshold = 0.5  # Define the overlap threshold

for imagefile in images_list:
    image_name = os.path.basename(imagefile)
    image_path = imagefile

    img = cv2.imread(image_path)
    img_height, img_width = img.shape[:2]  # Get image dimensions

    results = model(img, conf=0.3)

    blended_image = img.copy()  # Create a copy of the original image for blending
    masks_detected = False  # Flag to check if any masks were detected

    filtered_results = []
    for idx, result in enumerate(results):
        if result.masks is not None:
            boxes = result.boxes.xyxy.cpu().numpy().astype(int)
            confidences = result.boxes.conf.cpu().numpy()
            classes = result.boxes.cls.cpu().numpy().astype(int)
            class_names = result.names  # Get class names
            # Filter out overlapping boxes
            keep = np.ones(len(boxes), dtype=bool)
            for i in range(len(boxes)):
                if not keep[i]:
                    continue
                for j in range(i + 1, len(boxes)):
                    if calculate_iou(boxes[i], boxes[j]) > overlap_threshold:
                        if confidences[i] > confidences[j]:
                            keep[j] = False
                        else:
                            keep[i] = False

            filtered_boxes = boxes[keep]
            filtered_confidences = confidences[keep]
            filtered_classes = classes[keep]
            filtered_masks = result.masks.data[keep]

            filtered_results.append((filtered_boxes, filtered_confidences, filtered_classes, filtered_masks))

    for boxes, confidences, classes, masks in filtered_results:
        for i, mask in enumerate(masks):
            masks_detected = True  # Set flag to True if any mask is detected
            mask = (mask.cpu().numpy() * 255).astype("uint8")  # Move tensor to CPU and convert to binary
            mask_resized = cv2.resize(mask, (img_width, img_height), interpolation=cv2.INTER_NEAREST)
            
            # Find contours of the mask
            contours, _ = cv2.findContours(mask_resized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            if contours:
                # Approximate contours to reduce the number of points
                approx_contours = [cv2.approxPolyDP(cnt, epsilon=1.0, closed=True) for cnt in contours]
                
                for cnt in approx_contours:
                    for j in range(len(cnt)):
                        point = cnt[j][0]
                        distance_from_center = abs(point[0] - img_width / 2)
                        max_distance = img_width / 2
                        shift_factor = (distance_from_center / max_distance) * 38  # Gradient shift

                        if point[0] < img_width / 2:
                            # Point is on the left side of the image
                            shift_x = shift_factor
                        else:
                            # Point is on the right side of the image
                            shift_x = -shift_factor

                        shift_y = 0  # No vertical shift needed, adjust if necessary
                        M = np.float32([[1, 0, shift_x], [0, 1, shift_y]])
                        shifted_point = cv2.transform(np.array([[point]], dtype=np.float32), M)[0][0]
                        cnt[j][0] = shifted_point

                    # Connect endpoints of the contours to handle discontinuities
                    for j in range(len(cnt) - 1):
                        cv2.line(blended_image, tuple(cnt[j][0]), tuple(cnt[j + 1][0]), (0, 255, 0), 1)
                    cv2.line(blended_image, tuple(cnt[-1][0]), tuple(cnt[0][0]), (0, 255, 0), 1)
            
            class_name = class_names[classes[i]]
            color = colors[classes[i] % len(colors)]  # Get color for the class

            cv2.putText(
                blended_image,
                class_name,
                (point[0], point[1]),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                color,
                2,
                cv2.LINE_AA,
            )

    if masks_detected:
        output_path = os.path.join(output_dir, f"{os.path.splitext(image_name)[0]}_result.jpg")
        cv2.imwrite(output_path, blended_image)
        
        counter += 1
    else:
        print(f"No masks detected for image: {image_name}")
        

In [ ]:
# For Videos
import cv2
import numpy as np
import os
from ultralytics import YOLO

def calculate_iou(box1, box2):
    x1, y1, x2, y2 = box1
    x1_, y1_, x2_, y2_ = box2

    xi1 = max(x1, x1_)
    yi1 = max(y1, y1_)
    xi2 = min(x2, x2_)
    yi2 = min(y2, y2_)

    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2_ - x1_) * (y2_ - y1_)

    union_area = box1_area + box2_area - inter_area

    iou = inter_area / union_area
    return iou

lines_model = 'Models/Lanes_seg_v2.pt'
model = YOLO(lines_model)
input_video_path  = r"C:\Users\fbpza\Desktop\Road-Lines-Detection\Data\sample 2.mp4"
output_video_path = r'C:\Users\fbpza\Desktop\Road-Lines-Detection\Data\Detected\images\sample 2_resulted.mp4'


# input_video_path  = r"C:\Users\fbpza\Desktop\Road-Lines-Detection\Data\SV_video_1727690479.0518215.mp4"
# output_video_path = r'C:\Users\fbpza\Desktop\Road-Lines-Detection\Data\Detected\images\SV_video_17276904790518215_resulted.mp4'

cap = cv2.VideoCapture(input_video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

colors = [
    (255, 0, 0),  # Red
    (255, 0, 0),  # Green
    (0, 0, 255),  # Blue
    (255, 255, 0),  # Cyan
    (255, 0, 255),  # Magenta
    (0, 255, 255),  # Yellow
]

overlap_threshold = 0.5  # Define the overlap threshold

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    img_height, img_width = frame.shape[:2]  # Get frame dimensions

    results = model(frame, conf=0.3)

    blended_image = frame.copy()  # Create a copy of the original frame for blending
    masks_detected = False  # Flag to check if any masks were detected

    filtered_results = []
    for idx, result in enumerate(results):
        if result.masks is not None:
            boxes = result.boxes.xyxy.cpu().numpy().astype(int)
            confidences = result.boxes.conf.cpu().numpy()
            classes = result.boxes.cls.cpu().numpy().astype(int)
            class_names = result.names  # Get class names

            # Filter out overlapping boxes
            keep = np.ones(len(boxes), dtype=bool)
            for i in range(len(boxes)):
                if not keep[i]:
                    continue
                for j in range(i + 1, len(boxes)):
                    if calculate_iou(boxes[i], boxes[j]) > overlap_threshold:
                        if confidences[i] > confidences[j]:
                            keep[j] = False
                        else:
                            keep[i] = False

            filtered_boxes = boxes[keep]
            filtered_confidences = confidences[keep]
            filtered_classes = classes[keep]
            filtered_masks = result.masks.data[keep]

            filtered_results.append((filtered_boxes, filtered_confidences, filtered_classes, filtered_masks))

    for boxes, confidences, classes, masks in filtered_results:
        for i, mask in enumerate(masks):
            masks_detected = True  # Set flag to True if any mask is detected
            mask = (mask.cpu().numpy() * 255).astype("uint8")  # Move tensor to CPU and convert to binary
            mask_resized = cv2.resize(mask, (img_width, img_height), interpolation=cv2.INTER_NEAREST)
            
            
            # Find contours of the mask
            contours, _ = cv2.findContours(mask_resized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            if contours:
                # Approximate contours to reduce the number of points
                approx_contours = [cv2.approxPolyDP(cnt, epsilon=1.0, closed=True) for cnt in contours]
                
                for cnt in approx_contours:
                    for j in range(len(cnt)):
                        point = cnt[j][0]
                        distance_from_center = abs(point[0] - img_width / 2)
                        max_distance = img_width / 2
                        shift_factor = (distance_from_center / max_distance) * 38  # Gradient shift

                        if point[0] < img_width / 2:
                            # Point is on the left side of the image
                            shift_x = shift_factor
                        else:
                            # Point is on the right side of the image
                            shift_x = -shift_factor

                        shift_y = 0  # No vertical shift needed, adjust if necessary
                        M = np.float32([[1, 0, shift_x], [0, 1, shift_y]])
                        shifted_point = cv2.transform(np.array([[point]], dtype=np.float32), M)[0][0]
                        cnt[j][0] = shifted_point

                    # Connect endpoints of the contours to handle discontinuities
                    for j in range(len(cnt) - 1):
                        cv2.line(blended_image, tuple(cnt[j][0]), tuple(cnt[j + 1][0]), (0, 255, 0), 1)
                    cv2.line(blended_image, tuple(cnt[-1][0]), tuple(cnt[0][0]), (0, 255, 0), 1)
            
            class_name = class_names[classes[i]]
            color = colors[classes[i] % len(colors)]  # Get color for the class

            cv2.putText(
                blended_image,
                class_name,
                (point[0], point[1]),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                color,
                2,
                cv2.LINE_AA,
            )

    if masks_detected:
        out.write(blended_image)
    else:
        out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
# For Annotations(with drawings)
import cv2
import numpy as np
import glob, os
from ultralytics import YOLO

def calculate_iou(box1, box2):
    x1, y1, x2, y2 = box1
    x1_, y1_, x2_, y2_ = box2

    xi1 = max(x1, x1_)
    yi1 = max(y1, y1_)
    xi2 = min(x2, x2_)
    yi2 = min(y2, y2_)

    inter_area = max(0, xi2 - xi1) * max(0, yi1 - yi1)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2_ - x1_) * (y2_ - y1_)

    union_area = box1_area + box2_area - inter_area

    iou = inter_area / union_area
    return iou

lines_model = 'Models/Lanes_seg_v2.pt'
model = YOLO(lines_model)

counter = 0
images_src_dir = 'Data/Raw_images'
images_list = [i for i in glob.glob(images_src_dir + os.sep + "*.jpg")]
output_dir = 'Data/Detected/images'

os.makedirs(output_dir, exist_ok=True)

colors = [
    (255, 0, 0),  # Red
    (255, 0, 0),  # Green
    (0, 0, 255),  # Blue
    (255, 255, 0),  # Cyan
    (255, 0, 255),  # Magenta
    (0, 255, 255),  # Yellow
]

overlap_threshold = 0.5  # Define the overlap threshold

for imagefile in images_list:
    image_name = os.path.basename(imagefile)
    image_path = imagefile

    img = cv2.imread(image_path)
    img_height, img_width = img.shape[:2]  # Get image dimensions

    results = model(img, conf=0.3)

    blended_image = img.copy()  # Create a copy of the original image for blending
    masks_detected = False  # Flag to check if any masks were detected

    filtered_results = []
    yolo_format_lines = []  # List to store YOLOv8 format lines

    for idx, result in enumerate(results):
        if result.masks is not None:
            boxes = result.boxes.xyxy.cpu().numpy().astype(int)
            confidences = result.boxes.conf.cpu().numpy()
            classes = result.boxes.cls.cpu().numpy().astype(int)
            class_names = result.names  # Get class names
            # Filter out overlapping boxes
            keep = np.ones(len(boxes), dtype=bool)
            for i in range(len(boxes)):
                if not keep[i]:
                    continue
                for j in range(i + 1, len(boxes)):
                    if calculate_iou(boxes[i], boxes[j]) > overlap_threshold:
                        if confidences[i] > confidences[j]:
                            keep[j] = False
                        else:
                            keep[i] = False

            filtered_boxes = boxes[keep]
            filtered_confidences = confidences[keep]
            filtered_classes = classes[keep]
            filtered_masks = result.masks.data[keep]

            filtered_results.append((filtered_boxes, filtered_confidences, filtered_classes, filtered_masks))

    for boxes, confidences, classes, masks in filtered_results:
        for i, mask in enumerate(masks):
            masks_detected = True  # Set flag to True if any mask is detected
            mask = (mask.cpu().numpy() * 255).astype("uint8")  # Move tensor to CPU and convert to binary
            mask_resized = cv2.resize(mask, (img_width, img_height), interpolation=cv2.INTER_NEAREST)
            
            # Find contours of the mask
            contours, _ = cv2.findContours(mask_resized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            if contours:
                # Approximate contours to reduce the number of points
                approx_contours = [cv2.approxPolyDP(cnt, epsilon=1.0, closed=True) for cnt in contours]
                
                for cnt in approx_contours:
                    for j in range(len(cnt)):
                        point = cnt[j][0]
                        distance_from_center = abs(point[0] - img_width / 2)
                        max_distance = img_width / 2
                        shift_factor = (distance_from_center / max_distance) * 38  # Gradient shift

                        if point[0] < img_width / 2:
                            # Point is on the left side of the image
                            shift_x = shift_factor
                        else:
                            # Point is on the right side of the image
                            shift_x = -shift_factor

                        shift_y = 0  # No vertical shift needed, adjust if necessary
                        M = np.float32([[1, 0, shift_x], [0, 1, shift_y]])
                        shifted_point = cv2.transform(np.array([[point]], dtype=np.float32), M)[0][0]
                        cnt[j][0] = shifted_point

                    # Connect endpoints of the contours to handle discontinuities
                    for j in range(len(cnt) - 1):
                        cv2.line(blended_image, tuple(cnt[j][0]), tuple(cnt[j + 1][0]), (0, 255, 0), 1)
                    cv2.line(blended_image, tuple(cnt[-1][0]), tuple(cnt[0][0]), (0, 255, 0), 1)
                
                class_name = class_names[classes[i]]
                color = colors[classes[i] % len(colors)]  # Get color for the class

                cv2.putText(
                    blended_image,
                    class_name,
                    (point[0], point[1]),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    color,
                    2,
                    cv2.LINE_AA,
                )

                # Convert to YOLOv8 format
                x1, y1, x2, y2 = boxes[i]
                bbox_width = x2 - x1
                bbox_height = y2 - y1
                yolo_center_x = (x1 + x2) / 2 / img_width
                yolo_center_y = (y1 + y2) / 2 / img_height
                yolo_width = bbox_width / img_width
                yolo_height = bbox_height / img_height

                # Get the segmentation points
                for contour in contours:
                    points = contour.reshape(-1, 2)
                    normalized_points = [(x / img_width, y / img_height) for x, y in points]
                    formatted_points = " ".join([f"{x} {y}" for x, y in normalized_points])
                    yolo_format_lines.append(f"{classes[i]} {formatted_points}")

    if masks_detected:
        output_path = os.path.join(output_dir, f"{os.path.splitext(image_name)[0]}.jpg")
        cv2.imwrite(output_path, blended_image)

        # Save YOLOv8 format annotations
        txt_output_path = os.path.join(output_dir, f"{os.path.splitext(image_name)[0]}.txt")
        with open(txt_output_path, 'w') as f:
            f.write("\n".join(yolo_format_lines))
        
        counter += 1
    else:
        print(f"No masks detected for image: {image_name}")

In [ ]:
# For Annotations(with drawings)
import cv2
import numpy as np
import glob, os
from shutil import copyfile
from ultralytics import YOLO

def calculate_iou(box1, box2):
    x1, y1, x2, y2 = box1
    x1_, y1_, x2_, y2_ = box2

    xi1 = max(x1, x1_)
    yi1 = max(y1, y1_)
    xi2 = min(x2, x2_)
    yi2 = min(y2, y2_)

    inter_area = max(0, xi2 - xi1) * max(0, yi1 - yi1)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2_ - x1_) * (y2_ - y1_)

    union_area = box1_area + box2_area - inter_area

    iou = inter_area / union_area
    return iou

lines_model = 'Models/Lanes_seg_v2.pt'
model = YOLO(lines_model)

counter = 0
images_src_dir = 'Data/Raw_images'
images_list = [i for i in glob.glob(images_src_dir + os.sep + "*.jpg")]
output_dir = 'Data/Detected/images'

os.makedirs(output_dir, exist_ok=True)

overlap_threshold = 0.5  # Define the overlap threshold

for imagefile in images_list:
    image_name = os.path.basename(imagefile)
    image_path = imagefile

    img = cv2.imread(image_path)
    img_height, img_width = img.shape[:2]  # Get image dimensions

    results = model(img, conf=0.3)

    masks_detected = False  # Flag to check if any masks were detected

    filtered_results = []
    yolo_format_lines = []  # List to store YOLOv8 format lines

    for idx, result in enumerate(results):
        if result.masks is not None:
            boxes = result.boxes.xyxy.cpu().numpy().astype(int)
            confidences = result.boxes.conf.cpu().numpy()
            classes = result.boxes.cls.cpu().numpy().astype(int)
            class_names = result.names  # Get class names
            # Filter out overlapping boxes
            keep = np.ones(len(boxes), dtype=bool)
            for i in range(len(boxes)):
                if not keep[i]:
                    continue
                for j in range(i + 1, len(boxes)):
                    if calculate_iou(boxes[i], boxes[j]) > overlap_threshold:
                        if confidences[i] > confidences[j]:
                            keep[j] = False
                        else:
                            keep[i] = False

            filtered_boxes = boxes[keep]
            filtered_confidences = confidences[keep]
            filtered_classes = classes[keep]
            filtered_masks = result.masks.data[keep]

            filtered_results.append((filtered_boxes, filtered_confidences, filtered_classes, filtered_masks))

    for boxes, confidences, classes, masks in filtered_results:
        for i, mask in enumerate(masks):
            masks_detected = True  # Set flag to True if any mask is detected
            mask = (mask.cpu().numpy() * 255).astype("uint8")  # Move tensor to CPU and convert to binary
            mask_resized = cv2.resize(mask, (img_width, img_height), interpolation=cv2.INTER_NEAREST)
            
            # Find contours of the mask
            contours, _ = cv2.findContours(mask_resized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            if contours:
                # Approximate contours to reduce the number of points
                approx_contours = [cv2.approxPolyDP(cnt, epsilon=1.0, closed=True) for cnt in contours]
                
                for cnt in approx_contours:
                    for j in range(len(cnt)):
                        point = cnt[j][0]
                        distance_from_center = abs(point[0] - img_width / 2)
                        max_distance = img_width / 2
                        shift_factor = (distance_from_center / max_distance) * 38  # Gradient shift

                        if point[0] < img_width / 2:
                            # Point is on the left side of the image
                            shift_x = shift_factor
                        else:
                            # Point is on the right side of the image
                            shift_x = -shift_factor

                        shift_y = 0  # No vertical shift needed, adjust if necessary
                        M = np.float32([[1, 0, shift_x], [0, 1, shift_y]])
                        shifted_point = cv2.transform(np.array([[point]], dtype=np.float32), M)[0][0]
                        cnt[j][0] = shifted_point

                    # Convert to YOLOv8 format
                    x1, y1, x2, y2 = boxes[i]
                    bbox_width = x2 - x1
                    bbox_height = y2 - y1
                    yolo_center_x = (x1 + x2) / 2 / img_width
                    yolo_center_y = (y1 + y2) / 2 / img_height
                    yolo_width = bbox_width / img_width
                    yolo_height = bbox_height / img_height

                    # Get the segmentation points
                    for contour in contours:
                        points = contour.reshape(-1, 2)
                        normalized_points = [(x / img_width, y / img_height) for x, y in points]
                        formatted_points = " ".join([f"{x} {y}" for x, y in normalized_points])
                        yolo_format_lines.append(f"{classes[i]} {formatted_points}")

    if masks_detected:
        # Save YOLOv8 format annotations
        txt_output_path = os.path.join(output_dir, f"{os.path.splitext(image_name)[0]}.txt")
        with open(txt_output_path, 'w') as f:
            f.write("\n".join(yolo_format_lines))
        
        # Copy the original image to the output directory
        copyfile(image_path, os.path.join(output_dir, image_name))
        
        counter += 1
    else:
        print(f"No masks detected for image: {image_name}")